In [2]:
-- Select all stock items priced higher than the company-wide average
-- 1) ITEMS PRICED ABOVE AVERAGE

SELECT
    si.StockItemID,         -- show the item ID
    si.StockItemName,       -- show the item name
    si.UnitPrice            -- show the item price
FROM Warehouse.StockItems AS si   -- from the StockItems table
WHERE si.UnitPrice >              -- keep only items whose price is greater than
      (SELECT AVG(UnitPrice)      -- the average UnitPrice across all items
       FROM Warehouse.StockItems) -- subquery computes that overall average
ORDER BY si.UnitPrice DESC;       -- order results from most to least expensive

(27 rows affected)

Total execution time: 00:00:00.022

StockItemID,StockItemName,UnitPrice
215,Air cushion machine (Blue),1899.00
75,Ride on big wheel monster truck (Black) 1/12 scale,345.00
73,Ride on vintage American toy coupe (Red) 1/12 scale,285.00
74,Ride on vintage American toy coupe (Black) 1/12 scale,285.00
8,USB food flash drive - dim sum 10 drive variety pack,240.00
15,USB food flash drive - dessert 10 drive variety pack,240.00
174,Bubblewrap dispenser (Black) 1.5m,240.00
175,Bubblewrap dispenser (Blue) 1.5m,240.00
176,Bubblewrap dispenser (Red) 1.5m,240.00
67,Ride on toy sedan car (Black) 1/12 scale,230.00


In [4]:
-- Find customers whose delivery city is located in the United Kingdom
-- 2) Customers located in the United Kingdom

SELECT
    c.CustomerID,            -- show the customer ID
    c.CustomerName,          -- show the customer name
    c.DeliveryCityID         -- show where they receive goods
FROM Sales.Customers AS c    -- from the Customers table
WHERE c.DeliveryCityID IN (  -- keep only customers whose city ID is in this list:
    SELECT ci.CityID         -- get city IDs
    FROM Application.Cities AS ci
    JOIN Application.StateProvinces AS sp
        ON sp.StateProvinceID = ci.StateProvinceID   -- connect cities to their state/province
    WHERE sp.CountryID = (    -- filter by the United Kingdom
        SELECT CountryID
        FROM Application.Countries
        WHERE CountryName = N'United Kingdom'
    )
)
ORDER BY c.CustomerName;      -- sort alphabetically by customer


(0 rows affected)

Total execution time: 00:00:00.003

CustomerID,CustomerName,DeliveryCityID


In [6]:
-- For each customer, calculate their total order value using a correlated subquery
-- 3) Customers whose lifetime spend exceeds 10,000

SELECT TOP (30)
    c.CustomerID,   -- customer ID
    c.CustomerName, -- customer name
    (SELECT SUM(ol.Quantity * ol.UnitPrice)       -- total $ spent by this customer
     FROM Sales.Orders AS o
     JOIN Sales.OrderLines AS ol ON ol.OrderID = o.OrderID
     WHERE o.CustomerID = c.CustomerID) AS LifetimeSpend  -- connect subquery to outer row
FROM Sales.Customers AS c
WHERE (SELECT SUM(ol.Quantity * ol.UnitPrice)    -- filter: only customers with >10k total
       FROM Sales.Orders AS o
       JOIN Sales.OrderLines AS ol ON ol.OrderID = o.OrderID
       WHERE o.CustomerID = c.CustomerID) > 10000
ORDER BY LifetimeSpend DESC;  -- show biggest spenders first


(30 rows affected)

Total execution time: 00:00:00.054

CustomerID,CustomerName,LifetimeSpend
149,"Tailspin Toys (Inguadona, MN)",384393.35
132,"Tailspin Toys (Minidoka, ID)",379660.70
977,Mauno Laurila,377189.80
580,"Wingtip Toys (Sarversville, PA)",372350.00
964,Ingrida Zeltina,368067.45
14,"Tailspin Toys (Long Meadow, MD)",367258.50
954,Nasrin Omidzadeh,366883.75
593,"Wingtip Toys (Cuyamungue, NM)",365915.45
472,"Wingtip Toys (San Jacinto, CA)",365330.95
550,"Wingtip Toys (Morrison Bluff, AR)",360652.80


In [8]:
-- Use EXISTS to test if a customer has any orders during 2016
-- 4) Customers who placed orders in 2016

SELECT TOP (30)
    c.CustomerID,        -- show ID
    c.CustomerName       -- show name
FROM Sales.Customers AS c
WHERE EXISTS (           -- TRUE if the inner query finds at least one match
    SELECT 1             -- any constant works, since we only care about existence
    FROM Sales.Orders AS o
    WHERE o.CustomerID = c.CustomerID       -- link to outer customer
      AND o.OrderDate >= '2016-01-01'       -- starting Jan 1, 2016
      AND o.OrderDate <  '2017-01-01'       -- up to but not including 2017
)
ORDER BY c.CustomerName;  -- alphabetical order


(30 rows affected)

Total execution time: 00:00:00.026

CustomerID,CustomerName
832,Aakriti Byrraju
836,Abel Spirlea
869,Abel Tatarescu
1048,Abhra Ganguly
901,Adrian Andreasson
1055,Adriana Pena
1061,Agrita Abele
817,Agrita Kanepa
1034,Aishwarya Dantuluri
1016,Aive Petrov


In [10]:
-- Use a derived table to first compute each order's total, then filter by 5000+
-- 5) Derived table to find large orders

SELECT TOP (30)
    t.OrderID,          -- show order ID
    t.OrderTotal        -- show computed total
FROM (
    SELECT
        ol.OrderID,                             -- inner query groups by order
        SUM(ol.Quantity * ol.UnitPrice) AS OrderTotal  -- compute total value per order
    FROM Sales.OrderLines AS ol
    GROUP BY ol.OrderID                         -- needed when using SUM
) AS t                                            -- derived table alias
WHERE t.OrderTotal >= 5000                       -- keep only orders >= $5,000
ORDER BY t.OrderTotal DESC;                      -- sort from largest to smallest


(30 rows affected)

Total execution time: 00:00:00.036

OrderID,OrderTotal
30269,32026.00
66991,29536.00
49943,28688.60
47596,27640.00
69920,27382.00
72045,27366.00
40366,27306.00
61568,27161.00
17103,26787.00
17181,26645.00


In [11]:
-- Use a Common Table Expression (CTE) to compute order totals, then summarize by month
-- 6) Monthly sales totals using a CTE

WITH OrderTotals AS (                    -- name the CTE "OrderTotals"
    SELECT
        o.OrderID,                       -- each order ID
        o.OrderDate,                     -- when the order was made
        SUM(ol.Quantity * ol.UnitPrice) AS OrderTotal  -- calculate the total per order
    FROM Sales.Orders AS o
    JOIN Sales.OrderLines AS ol ON ol.OrderID = o.OrderID  -- join to get prices/qty
    WHERE o.OrderDate >= '2016-01-01' AND o.OrderDate < '2017-01-01'  -- only 2016 orders
    GROUP BY o.OrderID, o.OrderDate       -- group by order for SUM
)
SELECT
    DATEFROMPARTS(YEAR(OrderDate), MONTH(OrderDate), 1) AS SalesMonth,  -- extract month
    SUM(OrderTotal) AS MonthTotal        -- sum all orders per month
FROM OrderTotals
GROUP BY YEAR(OrderDate), MONTH(OrderDate)  -- one total per month
ORDER BY SalesMonth;                        -- show months in order


(5 rows affected)

Total execution time: 00:00:00.029

SalesMonth,MonthTotal
2016-01-01,4612140.45
2016-02-01,4099480.35
2016-03-01,4807110.70
2016-04-01,4739058.60
2016-05-01,5138002.65


In [13]:
-- Use multiple CTEs: one to get order totals, another for average per customer
-- 7) CTE to find customers with above-average order values

WITH OrderTotals AS (                          -- first CTE: order totals
    SELECT 
        o.OrderID,
        o.CustomerID,
        SUM(ol.Quantity * ol.UnitPrice) AS OrderTotal
    FROM Sales.Orders AS o
    JOIN Sales.OrderLines AS ol ON ol.OrderID = o.OrderID
    GROUP BY o.OrderID, o.CustomerID
),
CustomerAOV AS (                               -- second CTE: average order value per customer
    SELECT 
        CustomerID,
        AVG(OrderTotal) AS AvgOrderValue
    FROM OrderTotals
    GROUP BY CustomerID
)
SELECT TOP (20)
    c.CustomerID,          -- show customer ID
    c.CustomerName,        -- show name
    ca.AvgOrderValue       -- show their average order value
FROM CustomerAOV AS ca
JOIN Sales.Customers AS c ON c.CustomerID = ca.CustomerID  -- connect back to names
WHERE ca.AvgOrderValue > (SELECT AVG(OrderTotal) FROM OrderTotals)  -- above company avg
ORDER BY ca.AvgOrderValue DESC;   -- highest spenders first


(20 rows affected)

Total execution time: 00:00:00.087

CustomerID,CustomerName,AvgOrderValue
1058,Jaroslav Fisar,3896.600000
1056,Kalyani Benjaree,3446.803846
109,"Tailspin Toys (South Laguna, CA)",3358.154736
1036,Erik Malk,3322.660000
996,Laszlo Gardenier,3249.998181
945,Hoc Tran,3249.642708
40,"Tailspin Toys (Impact, TX)",3200.582038
977,Mauno Laurila,3143.248333
480,"Wingtip Toys (Wapinitia, OR)",3096.135652
1006,Taj Syme,3065.933009


In [15]:
-- CROSS APPLY runs a subquery per row and keeps only those with results
-- 8) CROSS APPLY — customers with their latest order only

SELECT
    c.CustomerID,               -- show customer ID
    c.CustomerName,             -- show name
    a.LastOrderID,              -- show most recent order ID
    a.LastOrderDate             -- show date of that order
FROM Sales.Customers AS c
CROSS APPLY (                   -- apply a per-customer subquery
    SELECT TOP (1)
        o.OrderID AS LastOrderID,     -- get one (latest) order
        o.OrderDate AS LastOrderDate
    FROM Sales.Orders AS o
    WHERE o.CustomerID = c.CustomerID  -- link subquery to current customer
    ORDER BY o.OrderDate DESC           -- sort so newest order is first
) AS a
ORDER BY a.LastOrderDate DESC;          -- list most recent customers first

SELECT TOP (30)
    c.CustomerID,               -- show customer ID
    c.CustomerName,             -- show name
    a.LastOrderID,              -- show most recent order ID
    a.LastOrderDate             -- show date of that order
FROM Sales.Customers AS c
CROSS APPLY (                   -- apply a per-customer subquery
    SELECT TOP (1)
        o.OrderID AS LastOrderID,     -- get one (latest) order
        o.OrderDate AS LastOrderDate
    FROM Sales.Orders AS o
    WHERE o.CustomerID = c.CustomerID  -- link subquery to current customer
    ORDER BY o.OrderDate DESC           -- sort so newest order is first
) AS a
ORDER BY a.LastOrderDate DESC;          -- list most recent customers first


(30 rows affected)

Total execution time: 00:00:00.143

CustomerID,CustomerName,LastOrderID,LastOrderDate
29,"Tailspin Toys (Eulaton, AL)",73557,2016-05-31
28,"Tailspin Toys (North Ridge, NY)",73519,2016-05-31
11,"Tailspin Toys (Devault, PA)",73573,2016-05-31
6,"Tailspin Toys (Jessie, ND)",73547,2016-05-31
35,"Tailspin Toys (Slanesville, WV)",73507,2016-05-31
64,"Tailspin Toys (Hodgdon, ME)",73561,2016-05-31
76,"Tailspin Toys (Yewed, OK)",73513,2016-05-31
82,"Tailspin Toys (La Cueva, NM)",73538,2016-05-31
87,"Tailspin Toys (Sauquoit, NY)",73522,2016-05-31
90,"Tailspin Toys (Tolna, ND)",73565,2016-05-31


In [17]:
-- OUTER APPLY is like a LEFT JOIN: still shows customers without any orders
-- 9) OUTER APPLY — show all customers even if no orders

SELECT TOP (30)
    c.CustomerID,               -- show customer ID
    c.CustomerName,             -- show name
    a.LastOrderID,              -- may be NULL if no order exists
    a.LastOrderDate
FROM Sales.Customers AS c
OUTER APPLY (                   -- same logic as above, but keeps unmatched rows
    SELECT TOP (1)
        o.OrderID AS LastOrderID,
        o.OrderDate AS LastOrderDate
    FROM Sales.Orders AS o
    WHERE o.CustomerID = c.CustomerID
    ORDER BY o.OrderDate DESC
) AS a
ORDER BY c.CustomerName;         -- sort alphabetically


(30 rows affected)

Total execution time: 00:00:00.127

CustomerID,CustomerName,LastOrderID,LastOrderDate
832,Aakriti Byrraju,73293,2016-05-27
836,Abel Spirlea,73243,2016-05-26
869,Abel Tatarescu,72420,2016-05-13
1048,Abhra Ganguly,73312,2016-05-27
901,Adrian Andreasson,73422,2016-05-28
1055,Adriana Pena,73318,2016-05-27
1061,Agrita Abele,73340,2016-05-27
817,Agrita Kanepa,73300,2016-05-27
1034,Aishwarya Dantuluri,73175,2016-05-26
1016,Aive Petrov,73310,2016-05-27


In [18]:
-- Show the top 10 items by total sales revenue using a nested derived table
-- 10) Derived table — Top 10 best-selling stock items

SELECT TOP (10)
    x.StockItemID,              -- show item ID
    x.StockItemName,            -- show item name
    x.TotalSales                -- show total revenue
FROM (
    SELECT
        si.StockItemID,                          -- inner query aggregates by item
        si.StockItemName,
        SUM(ol.Quantity * ol.UnitPrice) AS TotalSales  -- compute total $ per item
    FROM Sales.OrderLines AS ol
    JOIN Warehouse.StockItems AS si ON si.StockItemID = ol.StockItemID
    GROUP BY si.StockItemID, si.StockItemName
) AS x                                        -- name derived table "x"
ORDER BY x.TotalSales DESC;                   -- show highest-selling items first


(10 rows affected)

Total execution time: 00:00:00.010

StockItemID,StockItemName,TotalSales
215,Air cushion machine (Blue),11107251.00
173,32 mm Anti static bubble wrap (Blue) 50m,6384000.00
167,10 mm Anti static bubble wrap (Blue) 50m,6329070.00
161,20 mm Double sided bubble wrap 50m,6214320.00
164,32 mm Double sided bubble wrap 50m,6190240.00
158,10 mm Double sided bubble wrap 50m,5943000.00
170,20 mm Anti static bubble wrap (Blue) 50m,5795640.00
172,32 mm Anti static bubble wrap (Blue) 20m,2900160.00
219,Void fill 400 L bag (White) 400L,2871500.00
169,20 mm Anti static bubble wrap (Blue) 20m,2468700.00
